In [1]:
from textblob.classifiers import NaiveBayesClassifier
import numpy
import json
import nltk

def load_file(filename,t):
    f = open(filename,'rb')
    lines = f.read().splitlines()
    target = []
    if t == "int":
        for x in lines:
            target.append(int(x))
    else:
        if t == "json":
            for x in lines:
                y = json.loads(x)
                target.append(y)
        else:
            for x in lines:
                target.append(x)
    return target

collections = load_file("collections",'json')
originals = load_file("originals",'json')

In [2]:
def test_train_data(source,label):
    train=[]
    t1=[]
    t2=[]
    i=0
    data_size = len(source)
    numpy.random.seed(3)
    random_arr = numpy.random.choice(range(0,data_size),data_size)
    train_size = round(data_size*.64,1)
    devt_size = round(data_size*.16,1)
    target = []
    j = 0
    for i in random_arr:
        if j<train_size:
            target=train
        else:
            if j<train_size+devt_size:
                target=t1
            else:
                target=t2
        pair = (source[i],label)
        target.append(pair)
        j=j+1
    return (train,t1,t2)

c_tt = test_train_data(collections,"col")
o_tt = test_train_data(originals,"orig")

train = c_tt[0]+o_tt[0]

test1 = c_tt[1]+o_tt[1]
test2 = c_tt[2]+o_tt[2]

In [3]:
from HTMLParser import HTMLParser
import string
from textblob.utils import strip_punc

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def corpus_from_text(data,ls,key):
    #print(data)
    if data[key] is not None:        
        desc = strip_tags(data[key]).lower()
        desc = desc.split()
        for word in desc:
            word = strip_non_ascii(word)
            word = strip_punc(word,all=True)
            #print(word)
            if word not in ls:
                ls.append(word)
    return ls


In [7]:
def strip_non_ascii(string):
    stripped = (c for c in string if (0 < ord(c) < 127))
    return ''.join(stripped)

def first_words(text,num):
    ls = ""
    j = 0
    if text is not None:
        desc = strip_tags(text).lower()
        desc = desc.split()
        for word in desc:
            word = strip_non_ascii(word)
            word = strip_punc(word,all=True)
            #print(word)
            ls = ls + word
            j = j+1
            if j == num:
                return ls
        return ls
    return ls

In [8]:
manual_desc_corpus = ['reprints','collecting','hardcover','paperback','after','trade','as'
                     'into','vol']
manual_title_corpus = ['death','without','knight','four','line','by','collection','omnibus']
desc_corpus = manual_desc_corpus
title_corpus = manual_title_corpus
                
def basic_word_extractor(data,key):
    corpus = desc_corpus
    if key == 'name':
        corpus = title_corpus
    data_words = []
    data_words = corpus_from_text(data,data_words,key)
    features = dict((((key+'_contains({0})').format(word), (word in data_words))
                                            for word in corpus))
    return features

def volume_features(vol_data):
    #words_in_title = words(vol_data['name'])
    #words_in_desc = words(vol_data['name'])
    features= {'issue_count': vol_data['count_of_issues'],
            'start_year': int(vol_data['start_year']),
            'early_start': int(vol_data['start_year'])<=1990,
            'first_word': first_words(vol_data['description'],1),
            'first_two': first_words(vol_data['description'],2)
            #'name_contains(by)': 'by' in vol_data['name'].lower(),
            #'desc_contains(collected)': 'collected' in unicode(vol_data['description']).lower()
           }
    features.update(basic_word_extractor(vol_data,"name"))
    features.update(basic_word_extractor(vol_data,"description"))
    return features

In [9]:
train_feats = [(volume_features(n), category) for (n, category) in train]
devt_feats = [(volume_features(n), category) for (n, category) in test1]
test_feats = [(volume_features(n), category) for (n, category) in test2]
train2_feats = [(volume_features(n), category) for (n, category) in train+test1]
train_set = train2_feats
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [10]:
accuracy1 = nltk.classify.accuracy(classifier, test_feats)
print(accuracy1)
classifier.show_most_informative_features(10)

0.947368421053
Most Informative Features
description_contains(reprints) = True              col : orig   =     50.6 : 1.0
description_contains(paperback) = True              col : orig   =     26.3 : 1.0
description_contains(trade) = True              col : orig   =     16.6 : 1.0
description_contains(collecting) = True              col : orig   =     13.1 : 1.0
             early_start = True             orig : col    =     11.1 : 1.0
description_contains(hardcover) = True              col : orig   =     10.1 : 1.0
               first_two = u'seriesof'       col : orig   =      8.6 : 1.0
              start_year = 2003              col : orig   =      8.3 : 1.0
description_contains(vol) = True             orig : col    =      7.4 : 1.0
              first_word = u'hardcover'      col : orig   =      7.0 : 1.0


In [11]:
def prob_of_label(feats,cl):
    arr = {}
    dist = cl.prob_classify(feats)
    for label in dist.samples():
        row = "%s: %f" % (label, dist.prob(label))
        #print(row)
        arr[label]=dist.prob(label)
    return arr

## EX: prob_of_label(volume_features(collections[100]),classifier)

In [12]:
def trunc_desc(vol):
    if vol["description"] is not None:
        return vol["description"][0:30]
    else:
        return ""

errors = []
for (vol, tag) in test2:
    #print(vol['description'])
    if vol["name"] is not None:
        guess = classifier.classify(volume_features(vol))
        dist = prob_of_label(volume_features(vol),classifier)
        if guess != tag:
            errors.append( (tag, guess, vol["name"], json.dumps(vol['site_detail_url']), dist) )
        
for item in errors:
    print(item)

('col', 'orig', u'Iron Man Magazine', '"http://comicvine.gamespot.com/iron-man-magazine/4050-32600/"', {'col': 0.3324122319953169, 'orig': 0.6675877680046839})
('col', 'orig', u'Marvel Masterworks: Inhumans', '"http://comicvine.gamespot.com/marvel-masterworks-inhumans/4050-34836/"', {'col': 0.015653535850907296, 'orig': 0.9843464641490931})
('col', 'orig', u'Civil War: Amazing Spider-Man Decisions', '"http://comicvine.gamespot.com/civil-war-amazing-spider-man-decisions/4050-24551/"', {'col': 0.020124507434789444, 'orig': 0.9798754925652108})
('orig', 'col', u'New Thunderbolts', '"http://comicvine.gamespot.com/new-thunderbolts/4050-11298/"', {'col': 0.586006236492409, 'orig': 0.41399376350759226})
